In [1]:
####################
## Initial stuff
####################

# Load functions, modules and global variables required in our pipeline
from simulate_structures_functions import *
from importlib import reload  
from htmd.builder.charmm import _recoverProtonations

# Our main path
basepath = basepath = os.getcwd()+'/'
# Other Paths
strucpath = basepath + 'input_structures/'
resultspath = basepath + 'simulation_output/'
membranepdb = basepath + 'membrane/popc36_box_renumbered.pdb'
topparpath = basepath + 'toppar/TOP_PARAMS_ACE3/'#toppar= topology+parameters
ligandsdict_path = basepath + 'ligands.json'
modres_path = basepath + 'modified_residues.json'
slurmpath = basepath+'fake_slurm/'
# Path to slurm queing system binaries
# In our case, Ismael designed a bunch of small bash scripts (fake_slurm) which do ssh to Hydra and execute slurm there
# REmember to modify the "fake_slurm" commands by adding your username in them
path= os.environ['PATH']

# Modify path to include fake slurm
%env PATH=$path:$slurmpath

# USER INPUTS: introudce a json dictionary with the data for your system in the folowing format
"""
[
    {
        "name" : "NAME",
        "pdbfile" : "PATH/TO/YOUR_PDB.pdb", # Relative to basepath. NOT ABSOLUTE PATH
        "modres" : ["MODIFIED_RESNAME1", "MODIFIED_RESNAME"],
        "ligands" : [{
                        "resname":"SMALMOL_RESNAME1",
                        "name" : "SMALMOL_NAME1",
                        "covalently_bound" : true # If the SMALMOL is covalently bount to peptide
                    }],
        "pdbcode" : "PDBCODE", # CLosest-ressembling PDB structure of this GPCR
        "curated" : true, # If system has been already properly protonated
        "sod2x50" : true # If system requires addition of sodium near 2x50
    },
"""
input_dict = json_dict(basepath+"inputs.json")
# IMPORTANT: Peptide ligands must have L, L0, L1, L2 or PEP as its segid for 
#this pipeline to work properly
    
# Load topology, parameter and stream files with our current basepath
topos = [os.path.join(topparpath,file) for file in toposfilenames] 
params = [os.path.join(topparpath,file) for file in paramsfilenames]
streams = [os.path.join(topparpath,file) for file in streamsfilenames]


/home/david/miniconda3/lib/python3.6/site-packages/htmd/versionwarnings.py:28: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/david/miniconda3/lib/python3.6/site-packages/htmd/versionwarnings.py:34: UserWarning: As of HTMD 1.21 support for ACEMD v2 has stopped. Please use ACEMD3 instead as well as the corresponding equilibration and production protocols. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.21');`
  , UserWarning)
/home/david/miniconda3/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New stable HTMD version (1.26.5 python[3.9,<3.10.0a0,3.6,<3.7.0a0,3.8,<3.9.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.22.1).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.26.5 htmd=1.26.5 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



2022-01-11 08:55:54,107 - moleculekit.readers - WARNING - Element guessing failed for atom with name  DUM as the guessed element "D" was not found in the periodic table. Check for incorrect column alignment in the PDB file or report to moleculekit issue tracker.
2022-01-11 08:55:54,270 - moleculekit.readers - WARNING - Element guessing failed for atom with name  DUM as the guessed element "D" was not found in the periodic table. Check for incorrect column alignment in the PDB file or report to moleculekit issue tracker.


env: PATH=/home/david/bin:/home/david/.local/bin:/home/david/miniconda3/bin:/home/david/miniconda3/condabin:/home/david/bin:/home/david/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/david/bin:/usr/local/dowser/bin:/usr/local/dowser/bin/linux:/home/david/bin:/usr/local/dowser/bin:/usr/local/dowser/bin/linux:/gpcr/users/daranda/doctorat/PSYBIAS_simulations/fake_slurm/


In [2]:
########################################
# Part 1: Prepare parameters for ligands 
########################################

# Save mol2 files of ligand molecules and modres present in systems
(modresdict,ligandsdict,pdbfilesdict) = save_smalmol_mol2(input_dict, basepath)

# Get topology-parameter files for ligands
get_lig_toppar(ligandsdict, basepath, username, password, pdbfiles = pdbfilesdict)

# Get topology-parameter files for modified residues
get_modres_toppar(modresdict, basepath, username, password, pdbfiles = pdbfilesdict)

In [3]:
###########################
## Part 2: Build the models (PSYBIAS)
###########################

# Iterate by GPCRdb structures to simulate
for entry in input_dict:    
    try:

        # Entry's data
        name = entry['name']
        pdbcode = entry['pdbcode']
        pdbfile = basepath+entry['pdbfile']
        curated = entry['curated']
        sod = "sod_yes" if entry['sod2x50'] else "sod_no"
        
        # By default an apo and a complex version of the system will be build. Change at will
        for apo in [False,True]:
            
            #Starting simulation
            start_time = time.time()        
            sysname = name+'_apo' if apo else name
            mystrucpath = strucpath+sysname+'/'
            os.makedirs(mystrucpath, exist_ok=True)

            # Skip if there is already a model build for this
            if os.path.exists(resultspath+'build/'+sysname+'/structure.pdb'):
                print('Build model for '+sysname+' already exists. Skipping...')
                continue

            # Check if simulation is aminergic
            aminergic = gpcrdb_dict[pdbcode]['family'].startswith('001_001')
            adenosine = gpcrdb_dict[pdbcode]['family'].startswith('001_006_001')
            
            # Hydrate structures with dowser before anything else
            # DOWSER NEEDS TO BE INSTALLED FOR THIS STEP TO WORK
            watered_pdb = '%shydrated_%s.pdb'%(strucpath, name)
            dowser_waters(pdbfile,dowserpath,watered_pdb)
            
            ## Load structure
            #(sod2x50, watered_filename) = internal_waters(mystrucpath, pdbcode, gpcrdb_dict, apo, pdbfile,sod=sod)
            #gpcrdb_mol = Molecule(watered_filename)
            sod2x50 =True
            gpcrdb_mol = Molecule(watered_pdb)
                
            # If the pipeline is running in 'apoform mode', remove any non-protein, non-ion, non-water thing on the system      
            # Delete also sod2x50 (we don't want them here)
            if apo:
                gpcrdb_mol.remove('not (protein or water or ion) or element Na')
                
            # Remove unnecessary ligand molecules: mostly crystalization detergents, quelants, buffers,
            # or post-traductional glicosilations
            gpcrdb_mol.remove('resname '+' '.join(blacklist))
            
            # Remove 2x50Sodium from non-A-class GPCRs
            if not gpcrdb_dict[pdbcode]['family'].startswith('001'):
                gpcrdb_mol.remove('element NA')

            # Get aligned OPM structure
            thickness,opm_mol = get_opm(pdbcode)

            # Ismael's function to add labels (segid) for 'ligand' and 'protein' parts of the system
            gpcrdb_mol_fixed,prot_segids = fix_and_prepare_input(gpcrdb_mol,name,modresdict,new_pdb_chain)
            
            # If there's any, parameterize and rename covalent-bound ligands
            if not apo:
                (gpcrdb_mol_fixed, covligs) = covalent_ligands(gpcrdb_mol_fixed, name, ligandsdict)
            else:
                covligs = []
            
            # write file to remember to which chain in the original structure belongs each segment
            segchain_json(gpcrdb_mol_fixed, sysname, basepath, prot_segids)
            
            # Align structrues using sequences, and take first one
            alignment_results = sequenceStructureAlignment(gpcrdb_mol_fixed, opm_mol, maxalignments = 1)
            mol_aligned = alignment_results[0] 
            
            #Center to receptor XY
            center = np.mean(mol_aligned.get('coords',sel='chain P'),axis=0)
            mol_aligned.moveBy([-center[0],-center[1],0])
            
            # Prepare protein: asign titration states, flipping side chains of HIS, ASN and GLN; rotate some sidechains, optimize waters, etc.
            # Most of this is done with a HTMD function called proteinPrepare()
            # Skip step if we are working with curators structures
            prepared_mol = mol_aligned if curated else prepare_system(mol_aligned, pdbcode, thickness, sod2x50, aminergic, adenosine)
            
            #Add membrane
            print('Adding membrane...')
            membranemol = Molecule(membranepdb)
            mol_membraned, membrane_resnames, membrane_segids, xreps, yreps = add_membrane(prepared_mol, membranemol,prot_segids,membrane_distance)
            
            # Needed later for equilibration
            with open(mystrucpath+"const_sel.txt",'w') as out: 
                const_sel = 'segid '+' '.join(prot_segids)+' and name C CA N O or not (segid ' + \
                  ' '.join(prot_segids)+' or lipid or water or ions ) and noh or segid ION WAT and noh'
                out.write(const_sel)

            #Solvate
            print('Solvating...')
            mol_solvated = solvate_pdbmol(mol_membraned,membrane_segids,water_thickness,water_margin,buffer=buffer,coldist=coldist,prefix='WT')

            # Check if system has lone-pair hallogen atoms. If it does, use legacy CGenFF parameters
            has_halo = bool(len(gpcrdb_mol_fixed.get('name', 'element Br Cl I and not ion')))
            if has_halo:
                cgenff_par = [topparpath+'David_top_params/parameters/legacy_par_all36_cgenff.prm']
                cgenff_top = [topparpath+'David_top_params/topologies/legacy_top_all36_cgenff.rtf']
            else: 
                cgenff_par = [topparpath+'General_top_params/parameters/par_all36_cgenff.prm']
                cgenff_top = [topparpath+'General_top_params/topologies/top_all36_cgenff.rtf']
            
            #Obtain extra parameters for ligands and modified residues 
            ligstreams=extra_parameters(name, ligandsdict, modresdict, blacklist, covligs, basepath, has_halo)
            
            # Assignign terminology for cap atoms of protein chain, depending if it is the receptor protein or not
            caps = get_caps(prot_segids, mol_solvated)
            #e.g.: {'P0': ['first ACE', 'last CT3'], 'P1': ['first ACE', 'last CT3']}

            #Pre-build model
            print('Pre-build...')
            prebuildmol = charmm.build(mol_solvated, 
                                       topo=topos+cgenff_top, 
                                       param=params+cgenff_par,
                                       stream=streams+ligstreams,
                                       caps=caps,
                                       outdir=resultspath+'/pre-build/'+sysname,
                                       ionize=False)

            # Save prebuild model topologies in files, and  store prebuild model in molecule object
            prebuild_psffile = prebuildmol.topoloc
            prebuild_pdbfile = os.path.splitext(prebuildmol.topoloc)[0]+'.pdb'
            prebuildmol = Molecule(prebuild_pdbfile)
            _recoverProtonations(prebuildmol)

            # Checking of aromatic insertions (takes quite a lot fo time)
            print('Checking aromatic insertions...')
            mol_removed,removed_indexes = remove_aromatic_insertions(prebuildmol,prot_segids, outpdb=resultspath+'/pre-build/'+sysname+'/aromatic_check.pdb')

            # Checking of water/lipid ratio
            lipid_num = len(set(prebuildmol.get('resid',sel='segid '+membrane_lipid_segid)))
            solv_num = len(mol_removed.get('index',sel='resname TIP3 and name OH2'))
            if float(solv_num) / lipid_num < 35:
                raise ValueError('Water/lipid ratio lower than 35.')

            #Renumber residues
            print('Renumbering...')
            mol_renumbered = renumber_resid_vmd(mol_removed,'segid '+' '.join(membrane_segids),by=2)

            # Ionizing system
            print('Ionizing...')
            molbuilt = charmm.build(mol_removed,
                                    topo=topos+cgenff_top, 
                                    param=params+cgenff_par,
                                    stream=streams+ligstreams,                        
                                    outdir=resultspath+'/ionize/'+sysname,
                                    saltconc=0.15,
                                    caps=caps)
            build_psffile = molbuilt.topoloc
            build_pdbfile = os.path.splitext(molbuilt.topoloc)[0]+'.pdb'
            molbuilt = Molecule(build_pdbfile)
            _recoverProtonations(molbuilt)

            #Building system
            print('Building...')
            molbuilt = renumber_resid_vmd(molbuilt,'segid "WT.*" or segid I',by=2)
            molbuilt = charmm.build(molbuilt, 
                                    topo=topos+cgenff_top, 
                                    param=params+cgenff_par,
                                    stream=streams+ligstreams,                        
                                    outdir=resultspath+'/build/'+sysname,
                                    caps=caps,ionize=False)

            print('End of %s after %s seconds\n' % (sysname, time.time() - start_time))

    except Exception as e:
        print("model "+sysname+" could not be build because ",e)

2021-12-30 17:50:45,253 - moleculekit.molecule - INFO - Removed 0 atoms. 4887 atoms remaining in the molecule.
2021-12-30 17:50:47,136 - moleculekit.molecule - INFO - Removed 0 atoms. 4887 atoms remaining in the molecule.
2021-12-30 17:50:47,179 - moleculekit.molecule - INFO - Removed 3 atoms. 4884 atoms remaining in the molecule.
2021-12-30 17:50:47,276 - moleculekit.molecule - INFO - Removed 0 atoms. 4884 atoms remaining in the molecule.
2021-12-30 17:50:47,795 - moleculekit.tools.autosegment - INFO - Created segment P0 between resid 1 and 393.
2021-12-30 17:50:49,261 - moleculekit.molecule - INFO - Removed 0 atoms. 4884 atoms remaining in the molecule.
/home/david/miniconda3/lib/python3.6/site-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if yo

Adding membrane...


2021-12-30 17:50:57,256 - htmd.builder.builder - INFO - Replicating Membrane 3x3
Replicating Membrane: 100%|██████████| 9/9 [00:02<00:00,  3.87it/s]
2021-12-30 17:52:39,352 - moleculekit.molecule - INFO - Removed 2189 atoms. 42835 atoms remaining in the molecule.
2021-12-30 17:52:40,594 - moleculekit.molecule - INFO - Removed 126 residues from appended Molecule due to collisions.


Solvating...


2021-12-30 17:52:41,306 - htmd.builder.solvate - INFO - Using water pdb file at: /home/david/miniconda3/lib/python3.6/site-packages/htmd/share/solvate/wat.pdb


...

Building...


2021-12-30 18:16:32,198 - htmd.builder.charmm - INFO - Writing out segments.
2021-12-30 18:16:38,944 - htmd.builder.builder - INFO - 2 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 349, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 353, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 148, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 227, insertion: '', segid: 'P0'>



2021-12-30 18:16:39,984 - htmd.builder.charmm - INFO - Starting the build.
2021-12-30 18:16:48,150 - htmd.builder.charmm - INFO - Finished building.
2021-12-30 18:17:12,610 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3474, 3481, 3483, 3485]


End of NitroI_mut after 400.67685556411743 seconds

Build model for otava35 already exists. Skipping...
Build model for otava35_mut already exists. Skipping...
Build model for otava37 already exists. Skipping...
Build model for otava37_mut already exists. Skipping...


In [4]:
#########################
## Part 3: Equillibration
#########################

# Iterate by GPCRdb structures to simulate
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    for apo in [False]:
        # Entry's data
        try:
            modelname = name+'_apo' if apo else name
            equildir = resultspath+'equil/'+modelname+'/'
            if os.path.exists(equildir+'output.xtc') or os.path.exists(equildir+'simrunning'):
                print(" structure %s already has been equilibrated" % modelname)
                continue

            if not os.path.exists(equildir):
                os.makedirs(equildir)

            # Taking vmd selection line
            const_sel = []
            with open(strucpath+modelname+'/const_sel.txt', 'r') as outfile:
                const_sel += outfile.readlines()

            md = define_equilibration(const_sel)
            md.write(resultspath+'build/'+modelname,equildir)

            #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
            with open(equildir + 'run.sh', 'w') as f:
                f.write('#!/bin/bash\n%s > %slog.txt 2>&1' % (acemd_path, equildir))
        
            #Prepare slurm job  
            sq = SlurmQueue()
            sq.envvars = acemd_license
            sq.jobname = 'eql_'+modelname
            sq.datadir = None
            sq.partition = 'gpcr_gpu'
            sq.ngpu = 1
            sq.ncpu = 1
            sq.prerun = job_commands(equildir, '/home/daranda/'+modelname+'/')
            sq.memory = 2000
            sq.exclude = ['bifur','arwen','aragorn','dwalin','thorin','balin']
#             sq.nodelist = ['aragorn']
    
            # Submit
            sq.submit(equildir)
        except Exception as e:
            print("model "+modelname+" could not be send to equilibrate because of ",e)

2021-12-30 18:21:43,034 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3475, 3482, 3484, 3486]
2021-12-30 18:21:43,519 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2021-12-30 18:21:43,519 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2021-12-30 18:21:47,908 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/PSYBIAS_simulations/simulation_output/equil/metI/
2021-12-30 18:22:06,428 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3474, 3481, 3483, 3485]
2021-12-30 18:22:06,912 - htmd.protocols.equilibration

 structure otava35 already has been equilibrated


KeyboardInterrupt: 

In [3]:
#####################
## Part 4: Production
#####################

# Production protocol
md = define_production(timestep, trajperiod)

# For each structure 
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    for apo in [False]:
#     for apo in [True, False]:
        # must match with equildir in equilibration launcher code and contain input and output of equilibration.
        modelname = name+'_apo' if apo else name
        equildir = '%s/equil/%s/' % (resultspath, modelname)
        if not os.path.exists(equildir):
            print("structure %s has not been yet equillibrated. Skipping...")
            continue
        for rep in range(1,repnum+1):
            
            try: 
                # If simulation for this PDB has already been run
                proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
                if os.path.exists(proddir+'/output.xtc') or os.path.exists(proddir+'simrunning'):
                    print("replicate %d of structure %s already has been simulated" %(rep, modelname))
                    continue

                print('submitting replicate %d of %s' % (rep, modelname))
                # directory copy output of equilibration to production input (initial working directory for run_prod.sh).
                md.write(equildir,proddir)

                sq = SlurmQueue()
                sq.envvars = acemd_license
                sq.jobname = modelname+'_pr'+str(rep)
                sq.datadir = None
                sq.partition = 'gpcr_gpu'
                sq.prerun = job_commands(proddir, '/home/daranda/%s_pr_%d/'%(modelname,rep))
                sq.ngpu = 1
                sq.ncpu = 1
                sq.exclude = ['bifur','arwen','aragorn']

                #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
                with open(proddir + 'run.sh', 'w') as f:
                    f.write('#!/bin/bash\n%s >%slog.txt 2>&1' % (acemd_path,proddir))

                sq.submit(proddir)
                
            except Exception as e:
                print("model "+modelname+" could not be send to equilibrate because of ",e)            


replicate 1 of structure metI already has been simulated
replicate 2 of structure metI already has been simulated
replicate 3 of structure metI already has been simulated
replicate 1 of structure metI_mut already has been simulated
replicate 2 of structure metI_mut already has been simulated
replicate 3 of structure metI_mut already has been simulated
replicate 1 of structure NitroI already has been simulated
replicate 2 of structure NitroI already has been simulated
submitting replicate 3 of NitroI


2022-01-10 09:23:42,523 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3475, 3482, 3484, 3486]
2022-01-10 09:23:42,963 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-10 09:23:50,693 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/PSYBIAS_simulations/simulation_output/production/NitroI/rep_3/


submitting replicate 1 of NitroI_mut


2022-01-10 09:24:09,233 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3474, 3481, 3483, 3485]
2022-01-10 09:24:09,660 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-10 09:24:19,816 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/PSYBIAS_simulations/simulation_output/production/NitroI_mut/rep_1/


submitting replicate 2 of NitroI_mut


2022-01-10 09:24:36,454 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3474, 3481, 3483, 3485]
2022-01-10 09:24:36,902 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-10 09:24:43,123 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/PSYBIAS_simulations/simulation_output/production/NitroI_mut/rep_2/


submitting replicate 3 of NitroI_mut


2022-01-10 09:25:00,353 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3474, 3481, 3483, 3485]
2022-01-10 09:25:00,808 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-10 09:25:05,026 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/PSYBIAS_simulations/simulation_output/production/NitroI_mut/rep_3/


replicate 1 of structure otava35 already has been simulated
replicate 2 of structure otava35 already has been simulated
replicate 3 of structure otava35 already has been simulated
replicate 1 of structure otava35_mut already has been simulated
replicate 2 of structure otava35_mut already has been simulated
replicate 3 of structure otava35_mut already has been simulated
replicate 1 of structure otava37 already has been simulated
replicate 2 of structure otava37 already has been simulated
replicate 3 of structure otava37 already has been simulated
replicate 1 of structure otava37_mut already has been simulated
replicate 2 of structure otava37_mut already has been simulated
replicate 3 of structure otava37_mut already has been simulated


In [ ]:

##########################
## Part 5: Wrap Structures
##########################

# Wrap trajectories obtained during production with an htmd command
gpcr_sel = "protein and chain "+new_pdb_chain
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    for apo in [False]:
        try:
            modelname = name
            mymol_pdb = Molecule('%sproduction/%s/rep_1/structure.pdb' % (resultspath, modelname))
            for rep in range(1,repnum+1):
            #for rep in [3]:
                start_time = time.time()        
                print('wrapping replicate %d of %s' % (rep, modelname))
                proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
                rep = str(rep)

                # To avoid repeating wrapping in Trajectories already wrapped, check the existance of this file
                outname = proddir+'output_wrapped.xtc'
                if os.path.exists(outname):
                    print('replicate already wrapped. Skipping...')
                    continue
                
                # Wrap system
                mymol = Molecule('%sproduction/%s/rep_1/structure.psf' % (resultspath, modelname))
                mymol.read(proddir+'output.xtc')
                mymol.wrap(gpcr_sel)

                # Align frames and write
                mymol.align('all', refmol=mymol_pdb)
                mymol.write(outname)

            print('End of %s after %s seconds\n' % (modelname, time.time() - start_time))
        except Exception as e:
            print("model "+modelname+" could not be wrapped because of ",e)            




wrapping replicate 1 of metI
replicate already wrapped. Skipping...
wrapping replicate 2 of metI
replicate already wrapped. Skipping...
wrapping replicate 3 of metI
replicate already wrapped. Skipping...
End of metI after 0.014490127563476562 seconds

wrapping replicate 1 of metI_mut
replicate already wrapped. Skipping...
wrapping replicate 2 of metI_mut
replicate already wrapped. Skipping...
wrapping replicate 3 of metI_mut
replicate already wrapped. Skipping...
End of metI_mut after 0.011120319366455078 seconds

wrapping replicate 1 of NitroI
replicate already wrapped. Skipping...
wrapping replicate 2 of NitroI
replicate already wrapped. Skipping...
wrapping replicate 3 of NitroI
replicate already wrapped. Skipping...
End of NitroI after 0.010163545608520508 seconds

wrapping replicate 1 of NitroI_mut
replicate already wrapped. Skipping...
wrapping replicate 2 of NitroI_mut
replicate already wrapped. Skipping...
wrapping replicate 3 of NitroI_mut
replicate already wrapped. Skipping..

/home/david/miniconda3/lib/python3.6/site-packages/moleculekit/align.py:16: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, A))
  covariance = np.dot(P.T, Q)
/home/david/miniconda3/lib/python3.6/site-packages/moleculekit/align.py:54: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, C), array(float32, 2d, A))
  all1 = np.dot(all1, rot.T)


In [8]:
#####################
## EXTRA: ligand RMSD
#####################
### Assess quality of systems by calculating the RMSD of the ligand/s molecule/s

# Set paths and files
out = open(resultspath+'rmsd_lig.tsv','w')
out.write("Simulated_system\tRMSD_ligand\n")

# Iterate over pdbcodes 
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    # Iterate over production replicates
    for mytrajid in ["1","2","3"]:
        
#         try:
            sysname = name

            # Input files of simulation
            files_path = resultspath+'production/%s/rep_%s/'%(name, mytrajid)
            mypdbpath = files_path+'structure.pdb'
            mypsfpath = files_path+'structure.psf'
            mytrajpath = files_path+'output_wrapped.xtc'

            # Skip if no trajectory
            if not os.path.exists(mytrajpath):
                print("no trajectory replicate %s for system %s avalible. Skipping..."%(mytrajid, sysname))
                continue

            print('computing ligand RMSD for trajectory %s of system %s' % (mytrajid, sysname))

            # Load trajectory and topology into MDA universe, and select protein atoms
            u = mda.Universe(mypsfpath, mytrajpath)
            ligsel = u.select_atoms("segid LIG and not resname CLR")

            # Compute rmsd, extract its values and put them in corresponding lists and dicts
            R = rms.RMSD(ligsel)
            R.run()
            rmsd = np.mean([ a[2] for a in R.rmsd ])

            out.write("%s\t%f\n"%(sysname,rmsd)) 

#         except Exception as e:
#             print("error: system %s failed becasue %s"%(sysname,e))

computing ligand RMSD for trajectory 1 of system metI


2021-12-30 15:51:14,957 - MDAnalysis.analysis.rmsd - INFO - RMS calculation for 0 atoms.
2021-12-30 15:51:14,957 - MDAnalysis.analysis.base - INFO - Choosing frames to analyze
2021-12-30 15:51:14,958 - MDAnalysis.analysis.base - INFO - Starting preparation


ZeroDivisionError: float division